# Reviews Rephrasing - LLAMA

## Import

In [ ]:
%%capture
!pip install  transformers accelerate
!pip install --upgrade transformers accelerate

In [1]:
import pandas as pd
from transformers import AutoTokenizer
import transformers
import torch
from time import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown
import numpy as np
from tqdm import tqdm

## Huggingface login

In [2]:
from huggingface_hub import login
login(token="hf-api-key")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Read Files

In [3]:
from google.colab import drive
drive.mount('/content/drive')
movies = pd.read_json('/content/drive/My Drive/IMDB_movie_details_eng.json')
reviews = pd.read_json('/content/drive/My Drive/IMDB_reviews.json',lines = True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
movies.movie_id = movies.movie_id.astype('str')
reviews.movie_id = reviews.movie_id.astype('str')

df = pd.merge(movies, reviews, on='movie_id')

In [6]:
df_filtered = df[df.is_spoiler ==True]
# take one movie per genre
df_filtered = df_filtered.drop_duplicates(subset='genre', keep='first').reset_index(drop=True)
df_filtered.head()

,movie_id,plot_summary,duration,genre,rating_x,release_date,plot_synopsis,title,review_date,user_id,is_spoiler,review_text,rating_y,review_summary
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in...",Patriot Games,5 March 2008,ur16517420,True,The second Tom Clancy novel made into a film (...,6,decent Clancy thriller
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...,Last Vegas,2 June 2014,ur5291991,True,Last Vegas is a comedy that features an ensemb...,8,"Douglas,De Niro,Freeman and Kline Are Simply H..."
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,,Wet Hot American Summer,28 July 2015,ur3728510,True,"""Not ANOTHER 'Another Teen Movie,'"" is the acc...",8,Smells like teen spirit or is that the refrie...
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...,The Treasure of the Sierra Madre,27 June 2004,ur1406078,True,John Huston's genius as a director is undeniab...,10,"All that glitters is ""gone with the wind...."""
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...,Election,16 January 2009,ur8239592,True,"Popular, but frustrated high school civics tea...",9,A splendidly wicked satirical treat


## LLAMA 3

### pipeline

In [7]:
from transformers import pipeline
model = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline_rephrase = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Prompt Function

In [8]:
def rephrase_review(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024,
        verbose = False,
        pipeline = pipeline_rephrase
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    if verbose == True:
        return user_message + " " + answer  + " " +  ttime
    else:
        return answer




### Output Layout

In [38]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [39]:
summary = df_filtered.loc[93, 'plot_summary']
synopsis = df_filtered.loc[93, 'plot_synopsis']
review = df_filtered.loc[93, 'review_text']
system_message = f"""
You are an assistant specializing in spoiler detection for reviews. A spoiler is any information about the plot of a TV series, film, or book that could ruin the enjoyment for someone who has not yet seen or read it.
You have one task: 1) Rephrase the review to remove any spoilers while maintaining the original wording. To identify spoilers, consider any plot elements mentioned in the plot synopsis but not in the summary. Plot summary: {summary}, Plot synopsis: {synopsis}

"""
message = f"""'REVIEW': {review}"""

In [40]:
response = rephrase_review(
    system_message,
    user_message=message,
    temperature=0.001,
    max_length=1024,
    verbose = True)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 'REVIEW': OK, lets face it. We are over movies where one American kills 20 Germans who seemingly can't hit the wall of a Barn. They were big in the 1950's, and 1970s, but not any longer.Fury is a gritty film, with great productions values, really period vehicles, settings and costume. The acting is good, but it is let down by tow things.Clichéd characters. There is the tough commander, the new kid, the slightly crazy guy. There are a lot of clichéd characters here.Unrealistic battle scenes. Seriously I just switched off in the last scene. 20 or 30 SS guys with Panzerfasut would have destroyed an immobilied Sherman tank in 5 or 6 minutes. BUT 200 CRACK SS soldiers can't destroy a tank with its tracks off, sitting in the middle of a road????? The tank only has machine guns facing forward, so most of its sides are unprotected. Germans are carrying bazookas, but instead choose to shoot the tank with machine guns, which would not do anything? Seriously, one of the silliest, most unrealistic scenes in any war movie. Really spoilt what could have been a good movie. 

**<font color='green'>Answer:</font>** I've identified the spoilers in the review. Here's a rewritten version that maintains the original wording while removing any plot-specific information:

"REVIEW": OK, let's face it. We are over movies where one American kills 20 Germans who seemingly can't hit the wall of a Barn. They were big in the 1950's, and 1970s, but not any longer.

Fury is a gritty film, with great production values, really period vehicles, settings, and costume. The acting is good, but it is let down by two things.

Clichéd characters. There is the tough commander, the new kid, the slightly crazy guy. There are a lot of clichéd characters here.

Unrealistic scenes. I just couldn't get past the last scene. It was so over-the-top and unbelievable that I lost interest. The film's portrayal of war is just not convincing.

Overall, Fury had potential, but it was let down by its predictable characters and unrealistic scenes. 

**<font color='magenta'>Total time:</font>** 9.69 sec.

## Iterate on df

In [11]:
df_filtered['rephrased'] = np.nan
index = 0
for i in tqdm(range(93)):

      message = f""" 
      You are an assistant specializing in spoiler detection for reviews. A spoiler is any information about the plot of a TV series, film, or book that could ruin the enjoyment for someone who has not yet seen or read it.
      You have one task: 1) Rephrase the review to remove any spoilers while maintaining the original wording. To identify spoilers, consider any plot elements mentioned in the plot synopsis but not in the summary. Plot summary: {df_filtered.loc[i,'plot_summary']}, Plot synopsis: {df_filtered.loc[i,'plot_synopsis']}
      """
      df_filtered.loc[i, 'rephrased'] = rephrase_review(
      system_message = message,
      user_message= f"""REVIEW': {df_filtered.loc[i,'review_text']}""",
      temperature=0.001,
      max_length=1024)
      torch.cuda.memory_summary(device=None, abbreviated=False)
      torch.cuda.empty_cache()
      index += 1




100%|██████████| 93/93 [24:19<00:00, 15.69s/it]


In [12]:
print(df_filtered['rephrased'][1], '\n\n')
print(df_filtered['review_text'][1], '\n\n')

Here is a rewritten review that removes any spoilers while maintaining the original wording:

"Last Vegas is a comedy that features an ensemble cast of four legendary actors, including Michael Douglas, Robert De Niro, Morgan Freeman, and Kevin Kline. The film is about four old friends who have been friends for a long time and decide to take a trip together to celebrate a significant milestone. Mary Streenburgen co-stars. Directed by Jon Turteltaub, the movie promises to be a legendary experience, but unfortunately, it falls short of that mark. The Academy Award-winning actors seem to be working for a paycheck and having fun on set, but the comedic scenes fail to elicit much laughter from the viewer. One positive aspect is that the actors make the most of the material to ensure the movie is at least enjoyable. As for the performances, there's nothing particularly noteworthy, except that the actors seem to be having a good time with their characters. The screenplay is more like a celebra

In [34]:
df = df_filtered.iloc[:93]

In [35]:
df.to_csv('llama_reviews.csv')